# 2018 General Election Midterm Candidate Scraper

In [5]:
from urllib.request import urlopen
import re # Regex Library

In [2]:
def get_html_from_url(url): 
    return urlopen(url).read().decode("utf-8")

In [77]:
states = []
with open('states.txt', 'r') as f:
    for state in f:
        states.append(state[:-1].replace(' ', '_'))

In [78]:
states

['Alabama',
 'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Montana',
 'Nebraska',
 'Nevada',
 'New_Hampshire',
 'New_Jersey',
 'New_Mexico',
 'New_York',
 'North_Carolina',
 'North_Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'Rhode_Island',
 'South_Carolina',
 'South_Dakota',
 'Tennessee',
 'Texas',
 'Utah',
 'Vermont',
 'Virginia',
 'Washington',
 'West_Virginia',
 'Wisconsin',
 'Wyoming']

## House of Representatives

### Get Ballotpedia HTML for each state

In [84]:
state_general_election_html = {}
for state in states:
    html = get_html_from_url("https://ballotpedia.org/United_States_House_of_Representatives_elections_in_{},_2018".format(state))
    state_general_election_html[state] = re.findall(r"District \d{1,2}.+?General election candidates.+?Primary", html, re.DOTALL)

In [80]:
len(state_html)

50

In [6]:
ca = get_html_from_url("https://ballotpedia.org/United_States_House_of_Representatives_elections_in_California,_2018")

In [16]:
re.DOTALL

<RegexFlag.DOTALL: 16>

In [63]:
re.findall(r"District \d{1,2}", ca, re.DOTALL)

['District 1',
 'District 2',
 'District 3',
 'District 4',
 'District 5',
 'District 6',
 'District 7',
 'District 8',
 'District 9',
 'District 10',
 'District 11',
 'District 12',
 'District 13',
 'District 14',
 'District 15',
 'District 16',
 'District 17',
 'District 18',
 'District 19',
 'District 20',
 'District 21',
 'District 22',
 'District 23',
 'District 24',
 'District 25',
 'District 26',
 'District 27',
 'District 28',
 'District 29',
 'District 30',
 'District 31',
 'District 32',
 'District 33',
 'District 34',
 'District 35',
 'District 36',
 'District 37',
 'District 38',
 'District 39',
 'District 40',
 'District 41',
 'District 42',
 'District 43',
 'District 44',
 'District 45',
 'District 46',
 'District 47',
 'District 48',
 'District 49',
 'District 50',
 'District 51',
 'District 52',
 'District 53',
 'District 1',
 'District 2',
 'District 3',
 'District 4',
 'District 5',
 'District 6',
 'District 7',
 'District 8',
 'District 9',
 'District 10',
 'District

In [67]:
general_candidates = re.findall(r"District \d{1,2}.+?General election candidates.+?Primary", ca, re.DOTALL)

In [68]:
len(general_candidates)

53

In [86]:
ex = general_candidates[-10]

In [ ]:
re.findall

In [42]:
general_candidates

['</a> • <a href="/California%27s_2nd_Congressional_District_election,_2018" title="California\'s 2nd Congressional District election, 2018">District 2</a> • <a href="/California%27s_3rd_Congressional_District_election,_2018" title="California\'s 3rd Congressional District election, 2018">District 3</a> • <a href="/California%27s_4th_Congressional_District_election,_2018" title="California\'s 4th Congressional District election, 2018">District 4</a> • <a href="/California%27s_5th_Congressional_District_election,_2018" title="California\'s 5th Congressional District election, 2018">District 5</a> • <a href="/California%27s_6th_Congressional_District_election,_2018" title="California\'s 6th Congressional District election, 2018">District 6</a> • <a href="/California%27s_7th_Congressional_District_election,_2018" title="California\'s 7th Congressional District election, 2018">District 7</a>• <a href="/California%27s_8th_Congressional_District_election,_2018" title="California\'s 8th Congr

### Make dictionary with states as keys and values are lists of dictionaries with districts as keys and lists of candidates as values